Linear Algebra Assignment 4

In [18]:
import numpy as np
import networkx as nx
import random
import operator

Step: 1
- Compute the size of the graph, store it in a variable
- Compute the reverse of the graph, using method reverse() and an array "branching" such that "branching[i]"
    is the number of nodes linked to from node i. 
- Compute a list of dangling nodes
- Compute the initial approx of the ranking vector corresponding to x0 of PageNoteNotes

In [12]:
# Step 1: Setup
G = nx.read_edgelist('p2p-Gnutella08-mod.txt', comments='#',    # NB! Gnutella.txt must be in the directory as specified
                     create_using=nx.DiGraph(), 
                     delimiter='\t', 
                     nodetype=int, 
                     encoding='utf-8')

graph_size = G.size()
G_reversed = nx.DiGraph.reverse(G)       # returns the reverse of the graph, i.e. all the edges are reversed

# create an array "branching" such that branching[i] is the number of nodes linked to from node i, that is the out-degree of node i
branching = np.zeros(graph_size, dtype=int)
for idx, out_degree in G.out_degree():
    branching[idx] = out_degree

# find dangling nodes in the graph, that is nodes with out-degree 0
dangling_nodes = np.where(branching == 0)[0]


In [19]:
# Random Surfer implementation with networkx

def random_walk(damping_factor, num_iterations, wantNumOfVisits = False):
    # Select the initial random node 
    random_node = random.choice([i for i in range(G.number_of_nodes())])

    # initialize the initial count for all nodes as zero
    dict_counter = {}
    for i in range(G.number_of_nodes()):
        dict_counter[i] = 0

    # Since we start with initial node, this node gets a count of 1 to start with
    dict_counter[random_node] += 1

    # Traversing through the neighrbors of random node
    for i in range(num_iterations):     # num_iterations is the number of walks done in total
        m = random.random()
        list_of_neighbors = list(G.neighbors(random_node))
        if random_node in dangling_nodes:     # if the node is a dangling node, then jump to a random node
                random_node = random.choice([i for i in range(G.number_of_nodes())])
                dict_counter[random_node] += 1
        else:
            if m < 1-damping_factor:  # probablity walk to random neighbor
                random_node = random.choice(list_of_neighbors)   # select a random neighbor
                dict_counter[random_node] += 1
            else:       # jump to a random node, damping_factor% probability
                random_node = random.choice([i for i in range(G.number_of_nodes())])
                dict_counter[random_node] += 1
    
    # sort the dictionary by value in descending order
    sorted_random_walk = sorted(dict_counter.items(), key=operator.itemgetter(1), reverse=True)

    if wantNumOfVisits:
        return sorted_random_walk[:10]      # return the top 10 nodes with the most visits with their number of visits
    else:
        return list(map(operator.itemgetter(0), sorted_random_walk[:10]))     # return without the number of visits

In [14]:
# PageRank implementation with networkx (done naively)

def pageRankNaive(damping_factor, num_iterations):
    # compute mSx_k, this is done only once
    mSx_k = np.full(graph_size, 1/graph_size) * damping_factor   

    # initialize x_k0 as x_kx
    x_kx = np.full(graph_size, 1/graph_size) 

    for _ in range(num_iterations):
        # compute Dx_k:
        temp_sum_Dx = 0
        for i in range(graph_size):
            if i in dangling_nodes:
                temp_sum_Dx += x_kx[i]/graph_size
        Dx_k = np.full(graph_size, temp_sum_Dx)

        # compute Ax_k by looping over the backlinks to i:
        Ax_k = np.zeros(graph_size)
        for i in G.nodes():
            temp_sum_Ax = 0
            for j in G_reversed.out_edges(i):
                if branching[j[1]] != 0:
                    temp_sum_Ax += x_kx[j[1]]/branching[j[1]]
            Ax_k[i] = temp_sum_Ax

        # putting all together
        x_kx = (1-damping_factor)*Ax_k + (1-damping_factor)*Dx_k + mSx_k

    return (-x_kx).argsort()[:10]    # return the top 10 nodes with the highest PageRank


In [15]:
# print a list of the 10 highest ranked nodes in the PageRank algorithm for Gnutella graph
# Approx number of iterations of PageRank algorithm needed for the top 10 to stablise

for i in range(6):
    page_rank = pageRankNaive(0.15, i)  # 0.15 is the damping factor
    print(page_rank)

print("as you can see from result below, it takes 4 iterations for the top 10 to stablise")

[    0 13854 13853 13852 13851 13850 13849 13848 13855 13847]
[127 266 249 123 352 367 424 264 251 427]
[ 367  249  266  127  145  123  264  427  122 1317]
[ 367  249  145  266  264  127  123  122 1317    5]
[ 367  249  145  264  266  127  123  122 1317    5]
[ 367  249  145  264  266  123  127  122 1317    5]
as you can see from result below, it takes 4 iterations for the top 10 to stablise


In [20]:
# print a list of the 10 most visited nodes in the Random Surfer simulation for Gnutella graph

print(random_walk(0.15, 500_000, False))      # 0.15 is the damping factor, 500_000 walks, do not display number of visits for each node

print("it takes approximately 500_000 walks to get a result similar to the page rank algorithm")

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'G':[0m [1m[1mCannot determine Numba type of <class 'networkx.classes.digraph.DiGraph'>[0m
[1m
File "..\..\..\..\AppData\Local\Temp\ipykernel_22124\2092335367.py", line 6:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m